# This notebook is created to organize data from MetaCyc into one dataframe

### What kind of data is available by MetaCyc zipped file?
Noted that we recieved data from Pathway Tools platform version 22.6

**Important data that we extracted**
- compounds.dat
- enzrxns.dat
- reactions.dat

**Somewhat useful**
- atom-mapping and atom-mappint-smiles
- pathways

**May not be useful**
- classes
- gene_association
- genes
- metabolic-reactions.xml
- proteins
- protligandcplxes
- protseq
- pubs
- regulation
- rnas
- species
- transporters

In [1]:
import numpy as np
import pandas as pd
import math
import ast

### This is how our desired master dataframe look from last quarter

For now, make a dataset with enzyme/compounds linked to each other and then we can use RDKit-based function and existing functions to generate Mol-files, Distance, and Negative data

master_df = pd.read_csv('../datasets/MASTER_DF.csv')

master_df.head()

feature_df = master_df[['PubChem', 'dist', 'enzyme_class_1', 'enzyme_class_2', 'enzyme_class_3',
       'enzyme_class_4', 'enzyme_class_5', 'enzyme_class_6', 'enzyme_class_7',
        'n_O', 'n_N', 'n_P', 'n_S', 'n_X', 'DoU']]
feature_df.set_index(keys=['PubChem'], inplace=True)
feature_df.head()

## The current database from MetaCyc starts here

In [7]:
df_cpd = pd.read_csv('df_cpd.csv', index_col = 0)
df_rxn = pd.read_csv('parsed_rxns.csv', index_col = 0)
df_enz = pd.read_csv('df_enzrxns.csv', index_col = 0)
df_cpd = df_cpd.set_index(keys ='UNIQUE-ID')
df_rxn = df_rxn.set_index(keys = 'UNIQUE-ID')
df_enz = df_enz.set_index(keys = 'UNIQUE-ID')

# Here is an important note
The data from csv reading is in text formatted which lead to some problematic data handling.
- Some int data was read as float e.g. PubChemID which can be fixed by .astype() shown in cell #4
- List was formatted in text which required decoding by ast.literal_eval(x) which was written as a fucntion below

In [2]:
def recover_list(df, column):
    """This function will recover a list formatted string read from .csv into a list"""
    assert type(df[column][0]) != type([]), "TypeError: The data type is already a list, it should not be converted again"
    replacement = []
    for index, row in df.iterrows():
        
        data = []
        
        if type(row[column]) == type('string'):
            data = ast.literal_eval(row[column])
        else:
            pass
        replacement.append(data)
    df[column] = replacement
    return

In [9]:
# Change PubChemID into int type in df_cpd
PubChemID_int = df_cpd['PubChemID'].fillna(0).astype(int)
df_cpd['PubChemID'] = PubChemID_int

# Recover list format of df_rxn
rxn_list_fix = ['EC-NUMBER', 'ERXN-NUMBER', 'SUBSTRATES', 'PRODUCTS']
for col in rxn_list_fix:
    recover_list(df_rxn, col)

# Recover list format of df_enz
enz_list_fix = ['REACTION', 'ALTERNATIVE-SUBSTRATES', '^SUBSTRATE', 'KM', 'KCAT', 'VMAX']
for col in enz_list_fix:
    recover_list(df_enz, col)

##### the code below verify that all `.astype()` of PubChemID data didn't change the data unless it is `nan`
df_cpd = pd.read_csv('df_cpd.csv', index_col = 0)
df_cpd = df_cpd.set_index(keys ='UNIQUE-ID')
PubChemID_int = df_cpd['PubChemID'].fillna(0).astype(int)
for i in range(PubChemID_int.shape[0]):
    
#print (df_cpd['PubChemID'].loc['CPD-14743'])
#a = df_cpd['PubChemID'].loc['CPD-14743']
    if PubChemID_int[i] != df_cpd['PubChemID'][i]:
        print(PubChemID_int[i], df_cpd['PubChemID'][i])

In [11]:
df_cpd.head()

,COMMON-NAME,GIBBS-0,INCHI,SMILES,PubChemID
UNIQUE-ID,,,,,
CPD-14966,"(2Z,4Z)-2-hydroxy-5-carboxymuconate-6-semialde...",-150.733760,InChI=1S/C7H6O6/c8-3-4(6(10)11)1-2-5(9)7(12)13...,[CH](=O)C(C(=O)[O-])=CC=C(O)C([O-])=O,90657979
CPD-14905,"4&alpha;,14&alpha;-dimethyl-porifersta-8,25(27...",568.928300,InChI=1S/C31H52O/c1-9-23(20(2)3)11-10-21(4)24-...,CCC(C(C)=C)CCC(C)[CH]3(CCC4(C)(C2(CC[CH]1(C(C)...,102515047
CPD-14658,(+)-orobanchyl acetate,90.736885,InChI=1S/C21H24O7/c1-10-8-14(27-19(10)23)25-9-...,CC1(C(=O)OC(C=1)OC=C2(C(=O)O[CH]3([CH]2C(OC(C)...,24796587
CPD-14885,3-aminobenzoate,12.324662,InChI=1S/C7H7NO2/c8-6-3-1-2-5(4-6)7(9)10/h1-4H...,C(=O)([O-])C1(C=C(N)C=CC=1),3014145
CPD-14884,3-amino-4-hydroxybenzenesulfonate,-40.905334,"InChI=1S/C6H7NO4S/c7-5-3-4(12(9,10)11)1-2-6(5)...",C1(C(S(=O)([O-])=O)=CC(N)=C(O)C=1),4146016


In [12]:
df_rxn.head()

,ERXN-NUMBER,EC-NUMBER,SUBSTRATES,PRODUCTS,GIBBS
UNIQUE-ID,,,,,
RXN-12314,[ENZRXN-19061],[],"[CPD-7557, Red-NADPH-Hemoprotein-Reductases, O...","[CPD-13248, Ox-NADPH-Hemoprotein-Reductases, W...",-108.408860
RXN-16877,[ENZRXN-24025],[],[Cys-Cys-HlmE],[4N-2mthio-5oxo-3Spyrrolidine-2-COOH-HlmE],-29.530030
PORPHOBILSYNTH-RXN,"[PORPHOBILSYNTH-ENZRXN, ENZRXN66-1464, ENZRXN-...",[EC-4.2.1.24],[5-AMINO-LEVULINATE],"[PROTON, WATER, PORPHOBILINOGEN]",-33.708138
RXN-9204,[ENZRXN-14772],[EC-2.5.1.74],"[CPD-21340, DIHYDROXYNAPHTHOATE, PROTON]","[CPD-12118, PPI, CARBON-DIOXIDE]",-16.385834
RXN-9516,"[ENZRXN-15226, ENZRXN3O-10296, ENZRXN1G-511, E...","[EC-2.3.1.179, EC-2.3.1.41]","[Butanoyl-ACPs, MALONYL-ACP, PROTON]","[3-oxo-hexanoyl-ACPs, CARBON-DIOXIDE, ACP]",-6.862854


In [13]:
df_enz.head()

,COMMON-NAME,ENZYME,REACTION,ALTERNATIVE-SUBSTRATES,^SUBSTRATE,KM,KCAT,VMAX,PH-OPT,SPECIFIC-ACTIVITY,TEMPERATURE-OPT
UNIQUE-ID,,,,,,,,,,,
ENZRXN-15384,maleylacetate reductase,MONOMER-14207,[MALEYLACETATE-REDUCTASE-RXN],[],[],[],[],[],NaN,NaN,NaN
ENZRXN-26017,neurosporene &beta;-cyclase,MONOMER-20355,[RXN-8038],[],[],[],[],[],NaN,NaN,NaN
ENZRXN-18089,&beta;-<i>N</i>-acetylhexosaminidase,CPLX-8130,[3.2.1.52-RXN],[],[],[],[],[],NaN,NaN,NaN
ENZRXN-19053,cellulase,MONOMER-16501,[RXN-2043],"[(CELLULOSE |Lichenin|), (CELLULOSE |Xylogluca...",[],[],[],[],NaN,NaN,NaN
ENZRXN-952,IPP isomerase,YPL117C-MONOMER,[IPPISOM-RXN],[],[],[],[],[],NaN,NaN,NaN


`&beta;-<i>N</i>-acetylhexosaminidase` refer to __&beta;-<i>N</i>-acetylhexosaminidase__

Some weird notation here
- RXN-12314 doesn't have any enzyme affilated
- However, it links to a certain enzymatic reaction ENZRXN-19061 which has the reaction linked back as well
- What I don't understand is ENZYME column from enzrxn dataset which link to somewhere else that we cannot find an enzyme database in this current version of MetaCyc database
    - This ENZYME name and common name match with ENZYME name in rxn dataset. However, this is still unclear

##### Write a function that call a `UNIQUE-ID` or `COMMON-NAME` into InChI
This is an easiest approach to link df_rxn with df_cpd

In [45]:
def get_inchi(ID):
    
    """This function accept UNIQUE-ID and return InChI string of a certain compound"""
    
    inchi = df_cpd['INCHI'][ID]
    
    return inchi

def get_smiles(ID):
    
    """This function accept UNIQUE-ID and return SMILES string of a certain compound"""
    
    smiles = df_cpd['SMILES'][ID]
    
    return smiles

In [46]:
def get_pubchem(ID):
    
    """This function accept UNIQUE-ID and return InChI string of a certain compound"""
    if ID in df_cpd['PubChemID']:
        pubchem = df_cpd['PubChemID'][ID]
    else:
        pubchem = '0'
        
    return pubchem

In [195]:
get_inchi('CPD-7557')

'InChI=1S/C15H22O/c1-10-4-6-13-11(2)5-7-14(12(3)9-16)15(13)8-10/h8-9,11,13-15H,3-7H2,1-2H3/t11-,13+,14+,15+/m1/s1'

In [197]:
get_smiles('CPD-7557')

'CC1(CC[CH]2(C(CC[CH]([CH](C=1)2)C(C=O)=C)C))'

df_enz.loc['ENZRXN-19061']

type(int(df_cpd['PubChemID'].loc['CPD-14966']))

df_cpd.loc['5-AMINO-LEVULINATE']

df_cpd.loc['PROTON']

df_rxn.head()

df_rxn.loc['3.2.1.52-RXN']

To strip the string see method below
>>> import ast
>>> x = u'[ "A","B","C" , " D"]'
>>> x = ast.literal_eval(x)
>>> x
['A', 'B', 'C', ' D']
>>> x = [n.strip() for n in x]
>>> x
['A', 'B', 'C', 'D']

text = df_rxn['EC-NUMBER'].iloc[4]
text

type(text)

df_rxn.loc['PORPHOBILSYNTH-RXN']

text = df_rxn['EC-NUMBER'].loc['PORPHOBILSYNTH-RXN']

text = df_rxn['EC-NUMBER'].loc['RXN-9516']
split = text[2:-2].split("', '")
split

type(split)

split[0]

In [146]:
# count how many reactions doesn't have EC-NUMBER
counter_1 = 0
counter_m = 0
counter_n = 0

for index, row in df_rxn.iterrows():
    
    if type(row['EC-NUMBER']) != type('string'):
        #   if math.isnan(row['EC-NUMBER']):
        counter_n += 1
    else:
        data = ast.literal_eval(row['EC-NUMBER'])
        
        if len(data) == 1:
            counter_1 += 1
        elif len(data) > 1:
            counter_m += 1
        else:
            pass
print('Out of total', df_rxn.shape[0], 'row of df_rxn')
print('The data with only one, multiple, and no EC-Number are', counter_1, counter_m, 'and', counter_n, 'respectively')

Out of total 16834 row of df_rxn
The data with only one, multiple, and no EC-Number are 13251 200 and 3383 respectively


# Rearrange them into master dataframe format
Here, we have to turn all df_rxn into that of master dataframe by re-indexing with EC-Number as an index

In [44]:
df_rxn.head()

,ERXN-NUMBER,EC-NUMBER,SUBSTRATES,PRODUCTS,GIBBS
UNIQUE-ID,,,,,
RXN-12314,[ENZRXN-19061],[],"[CPD-7557, Red-NADPH-Hemoprotein-Reductases, O...","[CPD-13248, Ox-NADPH-Hemoprotein-Reductases, W...",-108.408860
RXN-16877,[ENZRXN-24025],[],[Cys-Cys-HlmE],[4N-2mthio-5oxo-3Spyrrolidine-2-COOH-HlmE],-29.530030
PORPHOBILSYNTH-RXN,"[PORPHOBILSYNTH-ENZRXN, ENZRXN66-1464, ENZRXN-...",[EC-4.2.1.24],[5-AMINO-LEVULINATE],"[PROTON, WATER, PORPHOBILINOGEN]",-33.708138
RXN-9204,[ENZRXN-14772],[EC-2.5.1.74],"[CPD-21340, DIHYDROXYNAPHTHOATE, PROTON]","[CPD-12118, PPI, CARBON-DIOXIDE]",-16.385834
RXN-9516,"[ENZRXN-15226, ENZRXN3O-10296, ENZRXN1G-511, E...","[EC-2.3.1.179, EC-2.3.1.41]","[Butanoyl-ACPs, MALONYL-ACP, PROTON]","[3-oxo-hexanoyl-ACPs, CARBON-DIOXIDE, ACP]",-6.862854


df_rxn['EC-NUMBER']['RXN-9204'][0]

df_rxn.iloc[0].index

# start running here

In [47]:
EC = []
rxn = []

for index, row in df_rxn.iterrows():
    
    if len(row['EC-NUMBER']) > 1:
        for i in range(len(row['EC-NUMBER'])):
            EC.append(row['EC-NUMBER'][i])
            rxn.append(index)
    elif len(row['EC-NUMBER']) == 1:
        EC.append(row['EC-NUMBER'][0])
        rxn.append(index)
    else:
        EC.append('No_Data')
        rxn.append(index)

In [48]:
df_master = pd.DataFrame({'EC-NUMBER' : EC,
                          'UNIQUE-ID' : rxn})

In [49]:
df_master.head()

,EC-NUMBER,UNIQUE-ID
0,No_Data,RXN-12314
1,No_Data,RXN-16877
2,EC-4.2.1.24,PORPHOBILSYNTH-RXN
3,EC-2.5.1.74,RXN-9204
4,EC-2.3.1.179,RXN-9516


In [66]:
rxn_num = []
subs = []
pdts = []
gibbs = []

for index, row in df_master.iterrows():
    ID = row['UNIQUE-ID']
    rxn_num.append(df_rxn['ERXN-NUMBER'][ID])
    subs.append(df_rxn['SUBSTRATES'][ID])
    pdts.append(df_rxn['PRODUCTS'][ID])
    gibbs.append(df_rxn['GIBBS'][ID])

In [67]:
df_master['ERXN-NUMBER'] = rxn_num
df_master['SUBSTRATES'] = subs
df_master['PRODUCTS'] = pdts
df_master['GIBBS'] = gibbs
df_master.head()

,EC-NUMBER,UNIQUE-ID,ERXN-NUMBER,SUBSTRATES,PRODUCTS,GIBBS
0,No_Data,RXN-12314,[ENZRXN-19061],"[CPD-7557, Red-NADPH-Hemoprotein-Reductases, O...","[CPD-13248, Ox-NADPH-Hemoprotein-Reductases, W...",-108.408860
1,No_Data,RXN-16877,[ENZRXN-24025],[Cys-Cys-HlmE],[4N-2mthio-5oxo-3Spyrrolidine-2-COOH-HlmE],-29.530030
2,EC-4.2.1.24,PORPHOBILSYNTH-RXN,"[PORPHOBILSYNTH-ENZRXN, ENZRXN66-1464, ENZRXN-...",[5-AMINO-LEVULINATE],"[PROTON, WATER, PORPHOBILINOGEN]",-33.708138
3,EC-2.5.1.74,RXN-9204,[ENZRXN-14772],"[CPD-21340, DIHYDROXYNAPHTHOATE, PROTON]","[CPD-12118, PPI, CARBON-DIOXIDE]",-16.385834
4,EC-2.3.1.179,RXN-9516,"[ENZRXN-15226, ENZRXN3O-10296, ENZRXN1G-511, E...","[Butanoyl-ACPs, MALONYL-ACP, PROTON]","[3-oxo-hexanoyl-ACPs, CARBON-DIOXIDE, ACP]",-6.862854


In [52]:
df_master.tail()

,EC-NUMBER,UNIQUE-ID,ERXN-NUMBER,SUBSTRATES,PRODUCTS,GIBBS
17109,EC-4.2.1.59,RXN-9537,"[ENZRXN0-7977, ENZRXN-21704, ENZRXN1G-803, ENZ...",[R-3-hydroxymyristoyl-ACPs],"[Tetradec-2-enoyl-ACPs, WATER]",22.705246
17110,No_Data,ARACHIDONATE-5-LIPOXYGENASE-RXN,[ENZRXN66-1526],"[OXYGEN-MOLECULE, ARACHIDONIC_ACID]",[6E8Z11Z14Z-5S-5-HYDROPEROXYCOSA-6],-69.979996
17111,EC-1.14.13,RXN-16742,[ENZRXN-23928],"[CPD-18060, NADPH, PROTON, OXYGEN-MOLECULE]","[CPD-18056, NADP, WATER]",-82.770510
17112,EC-3.4.23.50,RXN-11145,[],"[WATER, SQNYPIVQ-Cleavage-Sites]","[Mature-P17-Matrix, P24-Capsid-Proteins]",NaN
17113,EC-2.1.1.M46,RXN-19533,[ENZRXN-26293],"[S-ADENOSYLMETHIONINE, CPD-21063]","[CPD-21067, ADENOSYL-HOMO-CYS, PROTON]",12.328987


df_duplicate = df_master.groupby('EC-NUMBER').size().reset_index(name='count')
df_duplicate.head()

df_duplicate.tail()

In [68]:
df_sorted = df_master.sort_values(by=['EC-NUMBER'])
df_sorted.head()

,EC-NUMBER,UNIQUE-ID,ERXN-NUMBER,SUBSTRATES,PRODUCTS,GIBBS
2905,EC-1,RXN0-6277,"[ENZRXN-153, ENZRXN-483]","[CPD-722, Red-Thioredoxin]","[BIOTIN, Ox-Thioredoxin, WATER]",4.545235
7737,EC-1,RXN-8705,[],"[CPD-8922, OXYGEN-MOLECULE]",[CPD-8928],-50.319992
11493,EC-1,RXN-13817,[],"[CPD-8922, OXYGEN-MOLECULE, Donor-H2]","[CPD-14837, WATER, Acceptor]",NaN
16594,EC-1,R303-RXN,[ENZRXN-503],"[Nitroaromatic-Ox-Compounds, NADPH]","[Nitroaromatic-Red-Compounds, NADP]",NaN
16307,EC-1,RXN-9620,[],"[CPD-10257, OXYGEN-MOLECULE, Donor-H2]","[CPD-10258, WATER, Acceptor]",NaN


In [69]:
df_sorted.reset_index(inplace=True, drop=True)

In [70]:
df_sorted['GIBBS'][2]

nan

df_sorted['GIBBS'][2] = 'No-data'

In [71]:
for index, row in df_sorted.iterrows():
    
    if math.isnan(row['GIBBS']):
        df_sorted['GIBBS'][index] = 'No-Data'
        
df_sorted.head()

C:\Users\cholp\Miniconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


,EC-NUMBER,UNIQUE-ID,ERXN-NUMBER,SUBSTRATES,PRODUCTS,GIBBS
0,EC-1,RXN0-6277,"[ENZRXN-153, ENZRXN-483]","[CPD-722, Red-Thioredoxin]","[BIOTIN, Ox-Thioredoxin, WATER]",4.54523
1,EC-1,RXN-8705,[],"[CPD-8922, OXYGEN-MOLECULE]",[CPD-8928],-50.32
2,EC-1,RXN-13817,[],"[CPD-8922, OXYGEN-MOLECULE, Donor-H2]","[CPD-14837, WATER, Acceptor]",No-Data
3,EC-1,R303-RXN,[ENZRXN-503],"[Nitroaromatic-Ox-Compounds, NADPH]","[Nitroaromatic-Red-Compounds, NADP]",No-Data
4,EC-1,RXN-9620,[],"[CPD-10257, OXYGEN-MOLECULE, Donor-H2]","[CPD-10258, WATER, Acceptor]",No-Data


In [58]:
df_sorted['EC-NUMBER'][0]

'EC-1'

This cell might be wrong and lead to duplication of that EC-1

In [72]:
EC_a = 'EC-1'

EC = []
ID = []
erxn = []
subs = []
pdts = []
gibbs = []
counter = 0

ID_temp = []
erxn_temp = []
subs_temp = []
pdts_temp = []
gibbs_temp = []

for index, row in df_sorted.iterrows():
    
    if row['EC-NUMBER'] == EC_a:
        ID_temp.append(row['UNIQUE-ID'])
        erxn_temp.append(row['ERXN-NUMBER'])
        subs_temp.append(row['SUBSTRATES'])
        pdts_temp.append(row['PRODUCTS'])
        gibbs_temp.append(row['GIBBS'])
        counter += 1
        
    elif counter == 0:
        ID.append(row['UNIQUE-ID'])
        erxn.append(row['ERXN-NUMBER'])
        subs.append(row['SUBSTRATES'])
        pdts.append(row['PRODUCTS'])
        gibbs.append(row['GIBBS'])
        
        EC.append(EC_a)
        EC_a = row['EC-NUMBER']
    else:
        ID.append(ID_temp)
        erxn.append(erxn_temp)
        subs.append(subs_temp)
        pdts.append(pdts_temp)
        gibbs.append(gibbs_temp)
        
        ID_temp = []
        erxn_temp = []
        subs_temp = []
        pdts_temp = []
        gibbs_temp = []

        EC.append(EC_a)
        counter = 0
        EC_a = row['EC-NUMBER']


In [73]:
df_sorted_master = pd.DataFrame({'EC-NUMBER' : EC,
                                'UNIQUE-ID' : ID,
                                'ERXN-NUMBER' : erxn,
                                'SUBSTRATES' : subs,
                                'PRODUCTS' : pdts,
                                'GIBBS' : gibbs})

In [75]:
df_sorted_master.head(10)

,EC-NUMBER,UNIQUE-ID,ERXN-NUMBER,SUBSTRATES,PRODUCTS,GIBBS
0,EC-1,"[RXN0-6277, RXN-8705, RXN-13817, R303-RXN, RXN...","[[ENZRXN-153, ENZRXN-483], [], [], [ENZRXN-503...","[[CPD-722, Red-Thioredoxin], [CPD-8922, OXYGEN...","[[BIOTIN, Ox-Thioredoxin, WATER], [CPD-8928], ...","[4.5452347, -50.319992, No-Data, No-Data, No-D..."
1,EC-1.1,"[RXN-18132, RXN1G-1529, RXN-776, RXN-9318, RXN...","[[], [ENZRXN-15826], [], [], [], [ENZRXN-15825...","[[CPD-19423, Acceptor], [C86-cis-keto-mycolate...","[[CPD-19443, Donor-H2], [3-oxo-C86-cis-keto-my...","[No-Data, No-Data, No-Data, No-Data, No-Data, ..."
2,EC-1.1.1,"[KETOGAL-DEHYDROXY-RXN, RXN-13774, RXN-17376, ...","[[], [ENZRXN-20967], [], [], [ENZRXN1G-743], [...","[[D-galactopyranose, NADP], [CPD-14807, NADP],...","[[CPD-1242, NADPH, PROTON], [CPD-14806, NADPH,...","[-1.6264648000000002, -0.1665039, No-Data, 2.5..."
3,EC-1.1.1.1,"[RXN-10781, RXN-7694, RXN-12448, RXN-5424, RXN...","[[ENZRXN-16820], [ENZRXN3O-4113, ENZRXN3O-4112...","[[CPD-11671, NAD], [CPD-7033, NAD], [Secondary...","[[5-HYDROXYINDOLE_ACETALDEHYDE, NADH, PROTON],...","[2.541809, 2.541809, 21.624023, 2.541809, 2.54..."
4,EC-1.1.1.10,RXN-9633,"[ENZRXN1G-246, ENZRXN-15237]","[R-3-hydroxystearoyl-ACPs, NADP]","[3-oxo-stearoyl-ACPs, NADPH, PROTON]",20.3757
5,EC-1.1.1.100,"[RXN-10655, RXN1G-157, RXN-16626, RXN-9540, RX...","[[ENZRXN-16657], [ENZRXN3O-10712, ENZRXN1G-179...","[[3-hydroxy-cis-D7-tetraecenoyl-ACPs, NADP], [...","[[3-oxo-cis-D7-tetradecenoyl-ACPs, NADPH, PROT...","[20.375732, 20.375792999999998, 20.375732, 20...."
6,EC-1.1.1.101,"[ACYLGLYCERONE-PHOSPHATE-REDUCTASE-RXN, RXN-15...","[[ENZRXN-24829, ENZRXN3O-11401], [ENZRXN-22321...","[[1-PALMITOYLGLYCEROL-3-PHOSPHATE, NADP], [L-1...","[[PALMITOYLGLYCERONE-PHOSPHATE, NADPH, PROTON]...","[-0.16638184, -0.1665039, 20.375854]"
7,EC-1.1.1.102,"[3-DEHYDROSPHINGANINE-REDUCTASE-RXN, RXN-12642]","[[ENZRXN66-5214, ENZRXN-19565, ENZRXN-19561, E...","[[CPD-13612, NADP], [CPD-13611, NADP]]","[[DEHYDROSPHINGANINE, NADPH, PROTON], [CPD-136...","[-0.16648865, -0.16645813]"
8,EC-1.1.1.103,[RXN-14249],[[]],"[[THR, NAD]]","[[AMINO-ACETONE, CARBON-DIOXIDE, NADH]]",[-3.3482056000000004]
9,EC-1.1.1.104,RETINOL-DEHYDROGENASE-RXN,"[ENZRXN-19467, ENZRXN-19433]","[CPD-13524, NAD]","[RETINAL, NADH, PROTON]",0.991837


In [76]:
df_sorted_master.tail(10)

,EC-NUMBER,UNIQUE-ID,ERXN-NUMBER,SUBSTRATES,PRODUCTS,GIBBS
6842,|EC-7.5.2.i|,TRANS-RXN-333,[],"[D-galactofuransoe, ATP, WATER]","[D-galactofuransoe, ADP, Pi, PROTON]",-7.59888
6843,|EC-7.5.2.j|,ABC-33-RXN,[ABC-33-ENZRXN],"[ATP, WATER, D-Xylose]","[ADP, Pi, PROTON, D-Xylose]",-7.59882
6844,|EC-7.5.2.k|,TRANS-RXN-334,[],"[D-Galactose, ATP, WATER]","[D-Galactose, ADP, Pi, PROTON]",-7.59888
6845,|EC-7.5.2.l|,ABC-2-RXN,[ABC-2-ENZRXN],"[WATER, ATP, L-ARABINOSE]","[Pi, ADP, PROTON, L-ARABINOSE]",-7.59882
6846,|EC-7.5.2.m|,ABC-33-RXN,[ABC-33-ENZRXN],"[ATP, WATER, D-Xylose]","[ADP, Pi, PROTON, D-Xylose]",-7.59882
6847,|EC-7.5.2.n|,[ABC-2-RXN],[[ABC-2-ENZRXN]],"[[WATER, ATP, L-ARABINOSE]]","[[Pi, ADP, PROTON, L-ARABINOSE]]",[-7.598816]
6848,|EC-7.6.2.l|,3.6.3.38-RXN,[],"[CAPSULAR-POLYSACCHARIDES, WATER, ATP]","[CAPSULAR-POLYSACCHARIDES, Pi, ADP]",No-Data
6849,|EC-7.6.2.n|,ABC-56-RXN,[ENZRXN0-6365],"[ATP, Aliphatic-Sulfonates, WATER]","[Aliphatic-Sulfonates, ADP, Pi, PROTON]",-7.59888
6850,|EC-7.6.2.o|,ABC-32-RXN,[ABC-32-ENZRXN],"[ATP, THIAMINE, WATER]","[ADP, Pi, THIAMINE, PROTON]",-7.59888
6851,|EC-7.6.2.p|,ABC-25-RXN,"[ABC-25-ENZRXN, ABC-24A-ENZRXN]","[ATP, PUTRESCINE, WATER]","[ADP, Pi, PUTRESCINE, PROTON]",-7.59894


In [77]:
df_sorted_master.set_index(keys=['EC-NUMBER'], inplace=True)

In [78]:
df_sorted_master.index[4].count('.')

3

In [79]:
df_sorted_master

,UNIQUE-ID,ERXN-NUMBER,SUBSTRATES,PRODUCTS,GIBBS
EC-NUMBER,,,,,
EC-1,"[RXN0-6277, RXN-8705, RXN-13817, R303-RXN, RXN...","[[ENZRXN-153, ENZRXN-483], [], [], [ENZRXN-503...","[[CPD-722, Red-Thioredoxin], [CPD-8922, OXYGEN...","[[BIOTIN, Ox-Thioredoxin, WATER], [CPD-8928], ...","[4.5452347, -50.319992, No-Data, No-Data, No-D..."
EC-1.1,"[RXN-18132, RXN1G-1529, RXN-776, RXN-9318, RXN...","[[], [ENZRXN-15826], [], [], [], [ENZRXN-15825...","[[CPD-19423, Acceptor], [C86-cis-keto-mycolate...","[[CPD-19443, Donor-H2], [3-oxo-C86-cis-keto-my...","[No-Data, No-Data, No-Data, No-Data, No-Data, ..."
EC-1.1.1,"[KETOGAL-DEHYDROXY-RXN, RXN-13774, RXN-17376, ...","[[], [ENZRXN-20967], [], [], [ENZRXN1G-743], [...","[[D-galactopyranose, NADP], [CPD-14807, NADP],...","[[CPD-1242, NADPH, PROTON], [CPD-14806, NADPH,...","[-1.6264648000000002, -0.1665039, No-Data, 2.5..."
EC-1.1.1.1,"[RXN-10781, RXN-7694, RXN-12448, RXN-5424, RXN...","[[ENZRXN-16820], [ENZRXN3O-4113, ENZRXN3O-4112...","[[CPD-11671, NAD], [CPD-7033, NAD], [Secondary...","[[5-HYDROXYINDOLE_ACETALDEHYDE, NADH, PROTON],...","[2.541809, 2.541809, 21.624023, 2.541809, 2.54..."
EC-1.1.1.10,RXN-9633,"[ENZRXN1G-246, ENZRXN-15237]","[R-3-hydroxystearoyl-ACPs, NADP]","[3-oxo-stearoyl-ACPs, NADPH, PROTON]",20.3757
EC-1.1.1.100,"[RXN-10655, RXN1G-157, RXN-16626, RXN-9540, RX...","[[ENZRXN-16657], [ENZRXN3O-10712, ENZRXN1G-179...","[[3-hydroxy-cis-D7-tetraecenoyl-ACPs, NADP], [...","[[3-oxo-cis-D7-tetradecenoyl-ACPs, NADPH, PROT...","[20.375732, 20.375792999999998, 20.375732, 20...."
EC-1.1.1.101,"[ACYLGLYCERONE-PHOSPHATE-REDUCTASE-RXN, RXN-15...","[[ENZRXN-24829, ENZRXN3O-11401], [ENZRXN-22321...","[[1-PALMITOYLGLYCEROL-3-PHOSPHATE, NADP], [L-1...","[[PALMITOYLGLYCERONE-PHOSPHATE, NADPH, PROTON]...","[-0.16638184, -0.1665039, 20.375854]"
EC-1.1.1.102,"[3-DEHYDROSPHINGANINE-REDUCTASE-RXN, RXN-12642]","[[ENZRXN66-5214, ENZRXN-19565, ENZRXN-19561, E...","[[CPD-13612, NADP], [CPD-13611, NADP]]","[[DEHYDROSPHINGANINE, NADPH, PROTON], [CPD-136...","[-0.16648865, -0.16645813]"
EC-1.1.1.103,[RXN-14249],[[]],"[[THR, NAD]]","[[AMINO-ACETONE, CARBON-DIOXIDE, NADH]]",[-3.3482056000000004]


In [76]:
df_sorted_master.index

Index(['EC-1.1.1.1', 'EC-1.1.1.10', 'EC-1.1.1.100', 'EC-1.1.1.101',
       'EC-1.1.1.102', 'EC-1.1.1.103', 'EC-1.1.1.104', 'EC-1.1.1.105',
       'EC-1.1.1.106', 'EC-1.1.1.107',
       ...
       '|EC-7.5.2.i|', '|EC-7.5.2.j|', '|EC-7.5.2.k|', '|EC-7.5.2.l|',
       '|EC-7.5.2.m|', '|EC-7.5.2.n|', '|EC-7.6.2.l|', '|EC-7.6.2.n|',
       '|EC-7.6.2.o|', '|EC-7.6.2.p|'],
      dtype='object', name='EC-NUMBER', length=6682)

In [80]:
drop = []
for index, row in df_sorted_master.iterrows():
    if index.count('.') < 'EC-1.1.1.1'.count('.'):
        #print(index)
        drop.append(index)

In [81]:
drop[:10]

['EC-1',
 'EC-1.1',
 'EC-1.1.1',
 'EC-1.1.3',
 'EC-1.1.5',
 'EC-1.1.99',
 'EC-1.10.3',
 'EC-1.11.1',
 'EC-1.11.2',
 'EC-1.13.11']

In [82]:
df_sorted_master_drop = df_sorted_master
for item in drop:
    df_sorted_master_drop = df_sorted_master_drop.drop(item)

In [83]:
for index, row in df_sorted_master_drop.iterrows():
    if index.count('.') < 'EC-1.1.1.1'.count('.'):
        print(index)

In [85]:
df_sorted_master_drop.drop(['UNIQUE-ID', 'ERXN-NUMBER'], axis=1)

,SUBSTRATES,PRODUCTS,GIBBS
EC-NUMBER,,,
EC-1.1.1.1,"[[CPD-11671, NAD], [CPD-7033, NAD], [Secondary...","[[5-HYDROXYINDOLE_ACETALDEHYDE, NADH, PROTON],...","[2.541809, 2.541809, 21.624023, 2.541809, 2.54..."
EC-1.1.1.10,"[R-3-hydroxystearoyl-ACPs, NADP]","[3-oxo-stearoyl-ACPs, NADPH, PROTON]",20.3757
EC-1.1.1.100,"[[3-hydroxy-cis-D7-tetraecenoyl-ACPs, NADP], [...","[[3-oxo-cis-D7-tetradecenoyl-ACPs, NADPH, PROT...","[20.375732, 20.375792999999998, 20.375732, 20...."
EC-1.1.1.101,"[[1-PALMITOYLGLYCEROL-3-PHOSPHATE, NADP], [L-1...","[[PALMITOYLGLYCERONE-PHOSPHATE, NADPH, PROTON]...","[-0.16638184, -0.1665039, 20.375854]"
EC-1.1.1.102,"[[CPD-13612, NADP], [CPD-13611, NADP]]","[[DEHYDROSPHINGANINE, NADPH, PROTON], [CPD-136...","[-0.16648865, -0.16645813]"
EC-1.1.1.103,"[[THR, NAD]]","[[AMINO-ACETONE, CARBON-DIOXIDE, NADH]]",[-3.3482056000000004]
EC-1.1.1.104,"[CPD-13524, NAD]","[RETINAL, NADH, PROTON]",0.991837
EC-1.1.1.105,"[[CRPB-all-trans-Retinol, NAD]]","[[CRPB-all-trans-Retinal, NADH, PROTON]]",[3.2340088]
EC-1.1.1.106,"[PYRIDOXAL, NAD]","[CPD-375, NADH, PROTON]",-4.83823


This approach can be done by changing the substrates and products in df_rxn into PubChemID first ('0' for NaN)

In [86]:
# the file is too big!!! up to 1.7 G --- should reduce the size first
df_sorted_master.to_csv('df_master_1st.csv')

# changing to PubChemID still give 1.3 G left

# here is the first version of master dataframe
However, it is not perfect
- CPD-ID is not converted to PubChemID or InChI yet
- cofactor is not cleaned
- reversibility is not applied
- Anyway, should be in similar shape to that of previous master but contains single reactions

In [3]:
df_master_1st = pd.read_csv('df_master_1st.csv')

In [4]:
df_master = df_master_1st.drop(['UNIQUE-ID', 'ERXN-NUMBER'], axis = 1)
df_master['DIRECTION'] = 1
df_master.head()

,EC-NUMBER,SUBSTRATES,PRODUCTS,GIBBS,DIRECTION
0,EC-1,"[['CPD-722', 'Red-Thioredoxin'], ['CPD-8922', ...","[['BIOTIN', 'Ox-Thioredoxin', 'WATER'], ['CPD-...","[4.5452347, -50.319992, 'No-Data', 'No-Data', ...",1
1,EC-1.1,"[['CPD-19423', 'Acceptor'], ['C86-cis-keto-myc...","[['CPD-19443', 'Donor-H2'], ['3-oxo-C86-cis-ke...","['No-Data', 'No-Data', 'No-Data', 'No-Data', '...",1
2,EC-1.1.1,"[['D-galactopyranose', 'NADP'], ['CPD-14807', ...","[['CPD-1242', 'NADPH', 'PROTON'], ['CPD-14806'...","[-1.6264648000000002, -0.1665039, 'No-Data', 2...",1
3,EC-1.1.1.1,"[['CPD-11671', 'NAD'], ['CPD-7033', 'NAD'], ['...","[['5-HYDROXYINDOLE_ACETALDEHYDE', 'NADH', 'PRO...","[2.541809, 2.541809, 21.624023, 2.541809, 2.54...",1
4,EC-1.1.1.10,"['R-3-hydroxystearoyl-ACPs', 'NADP']","['3-oxo-stearoyl-ACPs', 'NADPH', 'PROTON']",20.375732,1


In [5]:
recover_list(df_master, 'SUBSTRATES')
recover_list(df_master, 'PRODUCTS')

In [6]:
df_master.head()

,EC-NUMBER,SUBSTRATES,PRODUCTS,GIBBS,DIRECTION
0,EC-1,"[[CPD-722, Red-Thioredoxin], [CPD-8922, OXYGEN...","[[BIOTIN, Ox-Thioredoxin, WATER], [CPD-8928], ...","[4.5452347, -50.319992, 'No-Data', 'No-Data', ...",1
1,EC-1.1,"[[CPD-19423, Acceptor], [C86-cis-keto-mycolate...","[[CPD-19443, Donor-H2], [3-oxo-C86-cis-keto-my...","['No-Data', 'No-Data', 'No-Data', 'No-Data', '...",1
2,EC-1.1.1,"[[D-galactopyranose, NADP], [CPD-14807, NADP],...","[[CPD-1242, NADPH, PROTON], [CPD-14806, NADPH,...","[-1.6264648000000002, -0.1665039, 'No-Data', 2...",1
3,EC-1.1.1.1,"[[CPD-11671, NAD], [CPD-7033, NAD], [Secondary...","[[5-HYDROXYINDOLE_ACETALDEHYDE, NADH, PROTON],...","[2.541809, 2.541809, 21.624023, 2.541809, 2.54...",1
4,EC-1.1.1.10,"[R-3-hydroxystearoyl-ACPs, NADP]","[3-oxo-stearoyl-ACPs, NADPH, PROTON]",20.375732,1


# Flattening the dataframe into a single liner
# start here

In [45]:
EC = []
substrate = []
products = []
gibbs = []
for index, row in df_master.iterrows():
    if type(row['SUBSTRATES']) == list:
        for item in row['SUBSTRATES']:
            EC.append(row['EC-NUMBER'])
            substrate.append(item)
        for item in row['PRODUCTS']:
            products.append(item)
        for item in row['GIBBS']:
            gibbs.append(item)

print ('EC-NUMBER count', len(EC), '\n SUBSTRATES count', len(substrate), '\n PRODUCTS count', len(products), '\n GIBBS count', len(gibbs))
            

EC-NUMBER count 21491 
 SUBSTRATES count 21491 
 PRODUCTS count 23154 
 GIBBS count 190856


# stop here

In [7]:
df_master_rev = pd.DataFrame({'EC-NUMBER': df_master['EC-NUMBER'],
                             'SUBSTRATES': df_master['PRODUCTS'],
                             'PRODUCTS': df_master['SUBSTRATES'],
                             'GIBBS': df_master['GIBBS'],
                             'DIRECTION': df_master['DIRECTION']*(-1)})
df_master_rev.head()

,EC-NUMBER,SUBSTRATES,PRODUCTS,GIBBS,DIRECTION
0,EC-1,"[[BIOTIN, Ox-Thioredoxin, WATER], [CPD-8928], ...","[[CPD-722, Red-Thioredoxin], [CPD-8922, OXYGEN...","[4.5452347, -50.319992, 'No-Data', 'No-Data', ...",-1
1,EC-1.1,"[[CPD-19443, Donor-H2], [3-oxo-C86-cis-keto-my...","[[CPD-19423, Acceptor], [C86-cis-keto-mycolate...","['No-Data', 'No-Data', 'No-Data', 'No-Data', '...",-1
2,EC-1.1.1,"[[CPD-1242, NADPH, PROTON], [CPD-14806, NADPH,...","[[D-galactopyranose, NADP], [CPD-14807, NADP],...","[-1.6264648000000002, -0.1665039, 'No-Data', 2...",-1
3,EC-1.1.1.1,"[[5-HYDROXYINDOLE_ACETALDEHYDE, NADH, PROTON],...","[[CPD-11671, NAD], [CPD-7033, NAD], [Secondary...","[2.541809, 2.541809, 21.624023, 2.541809, 2.54...",-1
4,EC-1.1.1.10,"[3-oxo-stearoyl-ACPs, NADPH, PROTON]","[R-3-hydroxystearoyl-ACPs, NADP]",20.375732,-1


In [8]:
df_merged =pd.concat([df_master, df_master_rev], ignore_index=True)
df_merged.head()

,EC-NUMBER,SUBSTRATES,PRODUCTS,GIBBS,DIRECTION
0,EC-1,"[[CPD-722, Red-Thioredoxin], [CPD-8922, OXYGEN...","[[BIOTIN, Ox-Thioredoxin, WATER], [CPD-8928], ...","[4.5452347, -50.319992, 'No-Data', 'No-Data', ...",1
1,EC-1.1,"[[CPD-19423, Acceptor], [C86-cis-keto-mycolate...","[[CPD-19443, Donor-H2], [3-oxo-C86-cis-keto-my...","['No-Data', 'No-Data', 'No-Data', 'No-Data', '...",1
2,EC-1.1.1,"[[D-galactopyranose, NADP], [CPD-14807, NADP],...","[[CPD-1242, NADPH, PROTON], [CPD-14806, NADPH,...","[-1.6264648000000002, -0.1665039, 'No-Data', 2...",1
3,EC-1.1.1.1,"[[CPD-11671, NAD], [CPD-7033, NAD], [Secondary...","[[5-HYDROXYINDOLE_ACETALDEHYDE, NADH, PROTON],...","[2.541809, 2.541809, 21.624023, 2.541809, 2.54...",1
4,EC-1.1.1.10,"[R-3-hydroxystearoyl-ACPs, NADP]","[3-oxo-stearoyl-ACPs, NADPH, PROTON]",20.375732,1


In [9]:
df_merged = df_merged.sort_values('EC-NUMBER').reset_index(drop = True)

In [10]:
df_merged.to_csv('df_merged_1st.csv')

In [11]:
df_merged.head()

,EC-NUMBER,SUBSTRATES,PRODUCTS,GIBBS,DIRECTION
0,EC-1,"[[CPD-722, Red-Thioredoxin], [CPD-8922, OXYGEN...","[[BIOTIN, Ox-Thioredoxin, WATER], [CPD-8928], ...","[4.5452347, -50.319992, 'No-Data', 'No-Data', ...",1
1,EC-1,"[[BIOTIN, Ox-Thioredoxin, WATER], [CPD-8928], ...","[[CPD-722, Red-Thioredoxin], [CPD-8922, OXYGEN...","[4.5452347, -50.319992, 'No-Data', 'No-Data', ...",-1
2,EC-1.1,"[[CPD-19423, Acceptor], [C86-cis-keto-mycolate...","[[CPD-19443, Donor-H2], [3-oxo-C86-cis-keto-my...","['No-Data', 'No-Data', 'No-Data', 'No-Data', '...",1
3,EC-1.1,"[[CPD-19443, Donor-H2], [3-oxo-C86-cis-keto-my...","[[CPD-19423, Acceptor], [C86-cis-keto-mycolate...","['No-Data', 'No-Data', 'No-Data', 'No-Data', '...",-1
4,EC-1.1.1,"[[D-galactopyranose, NADP], [CPD-14807, NADP],...","[[CPD-1242, NADPH, PROTON], [CPD-14806, NADPH,...","[-1.6264648000000002, -0.1665039, 'No-Data', 2...",1


In [12]:
subs_list = []
pdts_list = []

for index, row in df_merged.iterrows():
    
    Subs = []
    Pdts = []
    
    for item_set in row['SUBSTRATES']:
        if type(item_set) == list:
            for item in item_set:
                Subs.append(item)
        else:
            Subs.append(item_set)
    for item_set in row['PRODUCTS']:
        if type(item_set) == list:
            for item in item_set:
                Pdts.append(item)
        else:
            Pdts.append(item_set)
    subs_list.append(Subs)
    pdts_list.append(Pdts)
    
df_merged['SUBS'] = subs_list
df_merged['PDTS'] = pdts_list

In [13]:
f_subs = []
f_pdts = []
for item in subs_list:
    if type(item) == list:
        f_subs.append(item[0])
    else:
        f_subs.append(item)
for item in pdts_list:
    if type(item) == list:
        f_pdts.append(item[0])
    else:
        f_pdts.append(item)

In [14]:
df_merged['SUBS_1'] = f_subs
df_merged['PDTS_1'] = f_pdts

In [15]:
df_merged[df_merged['EC-NUMBER'] == 'EC-1.3.8.3']

,EC-NUMBER,SUBSTRATES,PRODUCTS,GIBBS,DIRECTION,SUBS,PDTS,SUBS_1,PDTS_1
3368,EC-1.3.8.3,"[ISOVALERYL-COA, ETF-Oxidized, PROTON]","[3-METHYL-CROTONYL-COA, ETF-Reduced]",7.090789999999999,1,"[ISOVALERYL-COA, ETF-Oxidized, PROTON]","[3-METHYL-CROTONYL-COA, ETF-Reduced]",ISOVALERYL-COA,3-METHYL-CROTONYL-COA
3369,EC-1.3.8.3,"[3-METHYL-CROTONYL-COA, ETF-Reduced]","[ISOVALERYL-COA, ETF-Oxidized, PROTON]",7.090789999999999,-1,"[3-METHYL-CROTONYL-COA, ETF-Reduced]","[ISOVALERYL-COA, ETF-Oxidized, PROTON]",3-METHYL-CROTONYL-COA,ISOVALERYL-COA


In [24]:
create_negative_matches(df_merged, 'EC-NUMBER', 'SUBS_1')

(              enzyme                                  product  reacts
 0       |EC-4.6.1.i|                           mRNA-Fragments     1.0
 1       |EC-4.6.1.i|                                    mRNAs     1.0
 2         EC-1.1.3.9                             MYO-INOSITOL     1.0
 3         EC-1.1.3.9                                  CPD-365     1.0
 4        EC-2.4.2.34  Flavonol-3-B-D-Xylosyl-12-B-D-Glycoside     1.0
 5        EC-2.4.2.34                  Flavonol-3-O-Glycosides     1.0
 6       EC-1.1.1.117                            Glucopyranose     1.0
 7       EC-1.1.1.117                            GLC-D-LACTONE     1.0
 8      EC-1.14.99.57                                PROTOHEME     1.0
 9      EC-1.14.99.57                                CPD-19716     1.0
 10      |EC-3.2.1.z|                                CPD-15456     1.0
 11      |EC-3.2.1.z|                                CPD-15453     1.0
 12      EC-2.1.1.148                        ADENOSYL-HOMO-CYS     1.0
 13   

The output above is sort-of a result of first substrate and product from each EC-NUBMER

It takes at least 10 minutes to generate all those 40 millions data but we will need to make the full dataframe

In [17]:
EC_NUM = []
Substrate = []
for index, row in df_merged.iterrows():
    for item in row['SUBS']:
        #print(df_merged['EC-NUMBER'][index], item)
        EC_NUM.append(df_merged['EC-NUMBER'][index])
        Substrate.append(item)

In [18]:
df_SingleLiner = pd.DataFrame({'EC': EC_NUM,
                       'SUBS': Substrate})
df_SingleLiner.to_csv('df_SingleLiner_1st.csv')

In [20]:
df_SingleLiner.tail()

,EC,SUBS
72773,|EC-7.6.2.p|,WATER
72774,|EC-7.6.2.p|,ADP
72775,|EC-7.6.2.p|,Pi
72776,|EC-7.6.2.p|,PUTRESCINE
72777,|EC-7.6.2.p|,PROTON


In [41]:
drop = []
for index, row in df_merged.iterrows():
    if row['EC-NUMBER'].count('.') < 'EC-1.1.1.1'.count('.'):
        #print(index)
        #drop.append(row['EC-NUMBER'])
        drop.append(index)
    elif row['EC-NUMBER'][0] == '|' and row['EC-NUMBER'][-1] == '|':
        #drop.append(row['EC-NUMBER'])
        drop.append(index)
    else:
        pass

In [42]:
df_dropped = df_merged
for item in drop:
    df_dropped = df_dropped.drop(item)

In [43]:
df_dropped.head()

,EC-NUMBER,SUBSTRATES,PRODUCTS,GIBBS,DIRECTION,SUBS,PDTS,SUBS_1,PDTS_1
6,EC-1.1.1.1,"[[CPD-11671, NAD], [CPD-7033, NAD], [Secondary...","[[5-HYDROXYINDOLE_ACETALDEHYDE, NADH, PROTON],...","[2.541809, 2.541809, 21.624023, 2.541809, 2.54...",1,"[CPD-11671, NAD, CPD-7033, NAD, Secondary-Alco...","[5-HYDROXYINDOLE_ACETALDEHYDE, NADH, PROTON, P...",CPD-11671,5-HYDROXYINDOLE_ACETALDEHYDE
7,EC-1.1.1.1,"[[5-HYDROXYINDOLE_ACETALDEHYDE, NADH, PROTON],...","[[CPD-11671, NAD], [CPD-7033, NAD], [Secondary...","[2.541809, 2.541809, 21.624023, 2.541809, 2.54...",-1,"[5-HYDROXYINDOLE_ACETALDEHYDE, NADH, PROTON, P...","[CPD-11671, NAD, CPD-7033, NAD, Secondary-Alco...",5-HYDROXYINDOLE_ACETALDEHYDE,CPD-11671
8,EC-1.1.1.10,"[R-3-hydroxystearoyl-ACPs, NADP]","[3-oxo-stearoyl-ACPs, NADPH, PROTON]",20.375732,1,"[R-3-hydroxystearoyl-ACPs, NADP]","[3-oxo-stearoyl-ACPs, NADPH, PROTON]",R-3-hydroxystearoyl-ACPs,3-oxo-stearoyl-ACPs
9,EC-1.1.1.10,"[3-oxo-stearoyl-ACPs, NADPH, PROTON]","[R-3-hydroxystearoyl-ACPs, NADP]",20.375732,-1,"[3-oxo-stearoyl-ACPs, NADPH, PROTON]","[R-3-hydroxystearoyl-ACPs, NADP]",3-oxo-stearoyl-ACPs,R-3-hydroxystearoyl-ACPs
10,EC-1.1.1.100,"[[3-hydroxy-cis-D7-tetraecenoyl-ACPs, NADP], [...","[[3-oxo-cis-D7-tetradecenoyl-ACPs, NADPH, PROT...","[20.375732, 20.375792999999998, 20.375732, 20....",1,"[3-hydroxy-cis-D7-tetraecenoyl-ACPs, NADP, R-3...","[3-oxo-cis-D7-tetradecenoyl-ACPs, NADPH, PROTO...",3-hydroxy-cis-D7-tetraecenoyl-ACPs,3-oxo-cis-D7-tetradecenoyl-ACPs


In [44]:
df_dropped.tail()

,EC-NUMBER,SUBSTRATES,PRODUCTS,GIBBS,DIRECTION,SUBS,PDTS,SUBS_1,PDTS_1
13315,EC-7.6.2.7,"[COBINAMIDE, ATP, WATER]","[COBINAMIDE, ADP, Pi, PROTON]",-7.5988809999999996,1,"[COBINAMIDE, ATP, WATER]","[COBINAMIDE, ADP, Pi, PROTON]",COBINAMIDE,COBINAMIDE
13316,EC-7.6.2.8,"[[ATP, COB-I-ALAMIN, WATER], [Cobalamins, WATE...","[[ADP, Pi, COB-I-ALAMIN, PROTON], [Cobalamins,...","[-7.598846400000001, 'No-Data']",1,"[ATP, COB-I-ALAMIN, WATER, Cobalamins, WATER, ...","[ADP, Pi, COB-I-ALAMIN, PROTON, Cobalamins, Pi...",ATP,ADP
13317,EC-7.6.2.8,"[[ADP, Pi, COB-I-ALAMIN, PROTON], [Cobalamins,...","[[ATP, COB-I-ALAMIN, WATER], [Cobalamins, WATE...","[-7.598846400000001, 'No-Data']",-1,"[ADP, Pi, COB-I-ALAMIN, PROTON, Cobalamins, Pi...","[ATP, COB-I-ALAMIN, WATER, Cobalamins, WATER, ...",ADP,ATP
13318,EC-7.6.2.9,"[[CPD-821, ATP, WATER]]","[[CPD-821, ADP, Pi, PROTON]]",[-7.598816],1,"[CPD-821, ATP, WATER]","[CPD-821, ADP, Pi, PROTON]",CPD-821,CPD-821
13319,EC-7.6.2.9,"[[CPD-821, ADP, Pi, PROTON]]","[[CPD-821, ATP, WATER]]",[-7.598816],-1,"[CPD-821, ADP, Pi, PROTON]","[CPD-821, ATP, WATER]",CPD-821,CPD-821


In [ ]:
holy_shit = create_negative_matches(df_test, 'EC', 'SUBS')

In [67]:
holy_shit[0]

NameError: name 'holy_shit' is not defined

In [68]:
holy_shit[1]

NameError: name 'holy_shit' is not defined

In [23]:
# from metamoles.py
def parse_reversible_reactions(reaction_list: list):
    """
    parse_reversible_reactions() queries the KEGG database with the input
        reaction list, and parses the results for all reactions that have been
        annotated with "<=>" in the reaction equation, which suggests that the
        catalyzed reaction is reversible

    Args:
        reaction_list (list): contains KEGG reaction IDs (e.g. 'R00709')

    Returns:
        list: contains KEGG IDs of reversible reactions
    """

    reversible_reaction = []
    for reaction in reaction_list:
        reaction_file = REST.kegg_get(reaction).read()
        for i in reaction_file.rstrip().split("\n"):
            if i.startswith("EQUATION") and "<=>" in i:
                reversible_reaction.append(reaction)
    return reversible_reaction

def combine_substrates_products(df: pd.DataFrame):
    """
    combine_substrates_products() is for use with a collection of enzymes
        in which it is understood that they are capable of catalyzing both the
        forward and reverser reactions. In this case, both the substrates and
        the products should be considered as bioreachable products.
        This function parses the list of substrates and products from their
        respective fields in the input dataframe, and returns a new dataframe
        with the combined substrates and products in a column labeled 'product'

    WARNING: combine_substrates_products() should not be run multiple times on
        the same dataframe becuase it will will append duplicate substrates

    Args:
        df (pandas.DataFrame): must contain fields
            ['entry', 'substrate', 'product']

    Returns:
        pandas.DataFrame: contains only fields ['entry', 'product']
    """

    rowindex = np.arange(0,len(df))
    df_with_ordered_index = df.set_index(rowindex)

    newdf = df_with_ordered_index
    # should this be a .copy()?

    for index,row in df_with_ordered_index.iterrows():
        productlist = row['product']
        substratelist = row['substrate']
        newdf.iloc[index,2] = productlist + substratelist

    return newdf[["entry","product"]]


def explode_dataframe(dataframe: pd.DataFrame, explosion_function,
                        explosion_target_field: str, fields_to_include: list):
    """
    explode_dataframe() applies the input explosion_function to the target
        field in each row of a dataframe. Each item in the output of the
        explosion_function is an anchor for a new row in the new dataframe. All
        of the supplied fields_to_include are added to the explosion item,
        and appended to the new dataframe row.

    Args:
        dataframe (pandas.DataFrame): input dataset
        explosion_function (function): function to be applied to target
            column in dataframe
        explosion_target_field (str): name of field in dataframe to which the
            explosion funciton will be applied
        fields_to_include (list): a list of strings that denote the columns of
            the input dataframe to be included in the output

    Returns:
        pandas.DataFrame: new exploded dataframe
    """
    new_rows = []
    for _, row in dataframe.iterrows():
        explosion_list = explosion_function(row[explosion_target_field])
        for item in explosion_list:
            row_data = [row[field] for field in fields_to_include]
            row_data.append(item)
            new_rows.append(row_data)

    fields_to_include.append(explosion_target_field)
    new_df = pd.DataFrame(new_rows, columns=fields_to_include)

    return new_df


def remove_cofactors(master_df: pd.DataFrame, master_cpd_field: str,
                     cofactor_df: pd.DataFrame, cofactor_field: str,
                     drop_na=True):
    """
    remove_cofactors() should be used to clean the dataset of cofactors. These
        will be included in the KEGG records as substrates and products, but
        are not actually products in the reaction

    Args:
        master_df (pandas.DataFrame): input dataset
        master_cpd_field (str): field that contains products
        cofactor_df (pandas.DataFrame): contains cofactors to be removed
        cofactor_field (str): field that contains cofactors
        drop_na (bool): default True

    Returns:
        pandas.DataFrame: cleaned data without cofactor entries
    """
    cofactor_list = parse_compound_ids(cofactor_df[cofactor_field])
    bool_mask = [False if cpd in cofactor_list else True for cpd in master_df[master_cpd_field]]
    clean_df = master_df[bool_mask]
    clean_df = clean_df.drop_duplicates()

    if drop_na:
        clean_df = clean_df[clean_df[master_cpd_field] != 'NA']
    else:
        pass

    return clean_df

def create_negative_matches(dataframe: pd.DataFrame,
                            enzyme_field: str, compound_field: str):
    """
    create_negative_matches() returns two dataframes.
        One dataframe is positive data that contains all the enzyme-compound
        pairs that exist in the input dataset.
        The second data frame is negative data made from matching all
        enzyme-compound pairs that do not exist in the dataset.

    Args:
        dataframe (pandas.DataFrame): input dataset
        enzyme_field (str): column in dataframe that contains enzyme ids
        compound_field (str): column in dataframe that contains compound ids

    Returns:
        pandas.DataFrame: positive data
            (contains fields ['enzyme', 'product', 'reacts'])
        pandas.DataFrame: negative data
            (contains fields ['enzyme', 'product', 'reacts'])
    """
    unique_enzymes = set(dataframe[enzyme_field].unique())
    # set of all unique enzymes in provided dataframes
    unique_cpds = set(dataframe[compound_field].unique())
    # set of all unique compounds in provided dataframe

    positive_data = []
    negative_data = []
    # initialize empty lists

    for enzyme in unique_enzymes:
    # iterate through unique enzyme set
        working_prods = set(dataframe[dataframe[enzyme_field] == enzyme][compound_field].unique())
        # unique set of all products reported to reaction with this enzyme in provided dataset
        non_working_prods = (unique_cpds - working_prods)
        # set math of all remaining products in the dataset minus those reported to react

        reactions = [{'reacts':1.0, 'enzyme':enzyme, 'product':product} for product in working_prods]
        # create new entry for each positive reaction
        non_reactions = [{'reacts':0.0, 'enzyme':enzyme, 'product':product} for product in non_working_prods]
        # create new entry for each negative reaction

        positive_data.extend(reactions)
        # add positive reactions to master list
        negative_data.extend(non_reactions)
        # add negative reactions to master list

    positive_df = pd.DataFrame(positive_data)
    negative_df = pd.DataFrame(negative_data)

    return positive_df, negative_df



#### Still, there is a problem
For some `EC-NUMBER` value, they are in `|_|` bracket with weird annotation with character instead of number. To solve this, we can modify some string value but if there is no repeating `EC-NUMBER` with and without bracket, we can just change the `EC-NUMBER` column without renewing the dataframe.

## So, we have a sortof Master Dataframe v1 right now

### What is the next step?
- Alter CPD-ID into PubChemID or InChI ---> go for PubChemID first
    - Done!
- Make reversible reaction
- Make negative dataset

In [5]:
def get_pubchem(ID):
    
    """This function accept UNIQUE-ID and return InChI string of a certain compound"""
    if ID in df_cpd['PubChemID']:
        pubchem = df_cpd['PubChemID'][ID]
    else:
        pubchem = '0'
        
    return pubchem

In [6]:
df_rxn.head()

,ERXN-NUMBER,EC-NUMBER,SUBSTRATES,PRODUCTS,GIBBS
UNIQUE-ID,,,,,
RXN-12314,[ENZRXN-19061],[],"[CPD-7557, Red-NADPH-Hemoprotein-Reductases, O...","[CPD-13248, Ox-NADPH-Hemoprotein-Reductases, W...",-108.408860
RXN-16877,[ENZRXN-24025],[],[Cys-Cys-HlmE],[4N-2mthio-5oxo-3Spyrrolidine-2-COOH-HlmE],-29.530030
PORPHOBILSYNTH-RXN,"[PORPHOBILSYNTH-ENZRXN, ENZRXN66-1464, ENZRXN-...",[EC-4.2.1.24],[5-AMINO-LEVULINATE],"[PROTON, WATER, PORPHOBILINOGEN]",-33.708138
RXN-9204,[ENZRXN-14772],[EC-2.5.1.74],"[CPD-21340, DIHYDROXYNAPHTHOATE, PROTON]","[CPD-12118, PPI, CARBON-DIOXIDE]",-16.385834
RXN-9516,"[ENZRXN-15226, ENZRXN3O-10296, ENZRXN1G-511, E...","[EC-2.3.1.179, EC-2.3.1.41]","[Butanoyl-ACPs, MALONYL-ACP, PROTON]","[3-oxo-hexanoyl-ACPs, CARBON-DIOXIDE, ACP]",-6.862854


In [16]:
'Nitroaromatic-Ox-Compounds' in df_cpd['PubChemID']

False

In [14]:
'BIOTIN' in df_cpd['PubChemID']

True

In [30]:
if 'BIOTIN' in df_cpd['PubChemID']:
    print('Yes')

Yes


In [39]:
get_pubchem('MALONYL-ACP')

'MALONYL-ACP'

In [7]:
# Start from df_rxn and rerun the master dataframe again
subs_id = []
pdts_id = []

for index, row in df_rxn.iterrows():
    
    subs = []
    for item in row['SUBSTRATES']:
        subs.append(get_pubchem(item))
    subs_id.append(subs)
    
    pdts = []
    for item in row['PRODUCTS']:
        pdts.append(get_pubchem(item))
    pdts_id.append(pdts)

In [8]:
df_rxn['SUBSTRATES'] = subs_id
df_rxn['PRODUCTS'] = pdts_id
df_rxn.head()

,ERXN-NUMBER,EC-NUMBER,SUBSTRATES,PRODUCTS,GIBBS
UNIQUE-ID,,,,,
RXN-12314,[ENZRXN-19061],[],"[15983961, 0, 977]","[52940136, 0, 962, 1038]",-108.408860
RXN-16877,[ENZRXN-24025],[],[0],[0],-29.530030
PORPHOBILSYNTH-RXN,"[PORPHOBILSYNTH-ENZRXN, ENZRXN66-1464, ENZRXN-...",[EC-4.2.1.24],[7048523],"[1038, 962, 6921588]",-33.708138
RXN-9204,[ENZRXN-14772],[EC-2.5.1.74],"[25244603, 54706667, 1038]","[45479709, 1023, 280]",-16.385834
RXN-9516,"[ENZRXN-15226, ENZRXN3O-10296, ENZRXN1G-511, E...","[EC-2.3.1.179, EC-2.3.1.41]","[0, 0, 1038]","[0, 280, 0]",-6.862854


In [45]:
counter_s_y = 0
counter_s_n = 0

counter_p_y = 0
counter_p_n = 0

for index, row in df_rxn.iterrows():
    
    for item in row['SUBSTRATES']:
        if item in df_cpd['PubChemID']:
            counter_s_y += 1
        else:
            counter_s_n += 1
    
    for item in row['PRODUCTS']:
        if item in df_cpd['PubChemID']:
            counter_p_y += 1
        else:
            counter_p_n += 1

print('Counting substrates True is', counter_s_y, 'False is', counter_s_n)
print('Counting products True is', counter_p_y, 'False is', counter_p_n)

Counting substrates True is 29253 False is 8287
Counting products True is 34480 False is 7752


In [49]:
percent_s = counter_s_y/(counter_s_y + counter_s_n)
percent_p = counter_p_y/(counter_p_y + counter_p_n)

print('Percents of available substrate and product are', percent_s*100, 'and', percent_p*100)

Percents of available substrate and product are 77.92488012786362 and 81.64425080507672


In [139]:
get_pubchem('Nitroaromatic-Ox-Compounds')

KeyError: 'Nitroaromatic-Ox-Compounds'

### Problems found
Where there is no data available for a certain UNIQUE-ID, the function goes wrong!!!
- Those are macromolecules, general group of compounds, etc.

Already solve by check availability by `in` function